# 数据预处理

## 单个数据预处理

为构建我们的本地知识库，我们需要对以多种类型存储的本地文档进行处理，读取本地文档并通过前文描述的 Embedding 方法将本地文档的内容转化为词向量来构建向量数据库。接下来以一些实际示例入手，来讲解如何对本地文档进行处理。
### 数据读取
将知识库源数据放置在 data_base/knowledge_db 目录下

**PDF文档为例**  
可以使用 LangChain 的 PyMuPDFLoader 来读取知识库的 PDF 文件。PyMuPDFLoader 是 PDF 解析器中速度最快的一种，结果会包含 PDF 及其页面的详细元数据，并且每页返回一个文档。

In [6]:
from langchain.document_loaders.pdf import PyMuPDFLoader

# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
loader = PyMuPDFLoader("data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf")

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pdf_pages = loader.load()

文档加载后储存在 `pdf_pages` 变量中:
- `pdf_pages` 的变量类型为 `List`
- 打印 `pdf_pages` 的长度可以看到 pdf 一共包含多少页

In [7]:
print(f"载入后的变量类型为：{type(pdf_pages)}，",  f"该 PDF 一共包含 {len(pdf_pages)} 页")

载入后的变量类型为：<class 'list'>， 该 PDF 一共包含 29 页


`pdf_pages` 中的每一元素为一个文档，变量类型为 `langchain_core.documents.base.Document`, 文档变量类型包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [8]:
pdf_page = pdf_pages[20]
print(f"每一个元素的类型：{type(pdf_page)}.", 
    f"该文档的描述性数据：{pdf_page.metadata}", 
    f"查看该文档的内容:\n{pdf_page.page_content}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'producer': 'Apache FOP Version 2.6', 'creator': 'DITA Open Toolkit', 'creationdate': '2023-05-23T21:45:33+08:00', 'source': 'data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'file_path': 'data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'total_pages': 29, 'format': 'PDF 1.4', 'title': '目录', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20230523214533+08'00'", 'page': 20}
------
查看该文档的内容:
7 可靠性设计
容错能力
ZXVMAX-S通过对关键软件资源的定时检测、实时任务监控、存储保护、数据校验、操作日志
信息保存等手段，可有效地防止小软件故障对系统所造成的冲击，提高了软件系统的容错能力
（即软件错误情况下的自愈能力）。
故障监视及处理
ZXVMAX-S具备自动检测与诊断系统软硬件故障的功能，可对故障硬件实施自动隔离、倒换、
重新启动、重新加载等处理。
满足漏洞扫描
为了避免存在安全漏洞使在网设备受外部攻击，需要使系统满足主流安全漏洞扫描工具的要求。
ZXVMAX-S满足Nessus和CD工具安全漏洞扫描的要求。
7.3 数据可靠性
ZXVMAX-S产品在不同分层架构下的数据都可以确保数据可靠和可恢复。

采集层
采集层运行环境部署采用冗余磁盘阵列设计，在服务器损坏、单一硬盘故障等场景下，通过
替换故障硬件后即可工作，数据不会丢失。同时，系统支持数据补充采集，确保系统数据连
续性。

存储共享层
存储共享层的MPP数据库，根据需要都设置了多

In [9]:
# 假设环境中缺少 NLTK 数据包中的 punkt 资源，这是 NLTK 库中用于分词的一个重要组件。解决这个问题的方法是按照报错信息中提到的步骤下载并安装 punkt 资源。
# import nltk
# nltk.download('punkt', quiet=True)

### 数据清洗

我们期望知识库的数据尽量是有序的、优质的、精简的，因此我们要删除低质量的、甚至影响理解的文本数据。  
可以看到上文中读取的pdf文件不仅将一句话按照原文的分行添加了换行符`\n`，也在原本两个符号中间插入了`\n`，我们可以使用正则表达式匹配并删除掉`\n`。

**处理单个page**

In [10]:
import re

# 预编译正则表达式（提升效率）
linebreak_pattern = re.compile(
    r'(?<![\\u4e00-\\u9fff])\n(?![\\u4e00-\\u9fff])',  # 负向断言匹配非中文环境换行
    flags=re.DOTALL
)
space_pattern = re.compile(r'[ 　]+')  # 匹配半角/全角空格
special_chars = ['•', '▪', '▫', '▶', '®', '©']  # 可扩展的干扰符号列表

In [11]:
pdf_page.page_content = re.sub(
        linebreak_pattern,
        lambda m: m.group().replace('\n', ''),
        pdf_page.page_content
    )
    
# 2. 批量清理特殊符号
for char in special_chars:
        pdf_page.page_content = pdf_page.page_content.replace(char, '')
    
# 3. 安全删除空格（保留URL等特殊场景）
pdf_page.page_content = space_pattern.sub('', pdf_page.page_content)

pdf_page.page_content

'7可靠性设计容错能力\nZXVMAX-S通过对关键软件资源的定时检测、实时任务监控、存储保护、数据校验、操作日志信息保存等手段，可有效地防止小软件故障对系统所造成的冲击，提高了软件系统的容错能力（即软件错误情况下的自愈能力）。故障监视及处理\nZXVMAX-S具备自动检测与诊断系统软硬件故障的功能，可对故障硬件实施自动隔离、倒换、重新启动、重新加载等处理。满足漏洞扫描为了避免存在安全漏洞使在网设备受外部攻击，需要使系统满足主流安全漏洞扫描工具的要求。\nZXVMAX-S满足Nessus和CD工具安全漏洞扫描的要求。\n7.3\xa0数据可靠性\nZXVMAX-S产品在不同分层架构下的数据都可以确保数据可靠和可恢复。\uf06c采集层采集层运行环境部署采用冗余磁盘阵列设计，在服务器损坏、单一硬盘故障等场景下，通过替换故障硬件后即可工作，数据不会丢失。同时，系统支持数据补充采集，确保系统数据连续性。\uf06c存储共享层存储共享层的MPP数据库，根据需要都设置了多个冗余备份，单一节点的损坏不会影响整系统的运行，也不会导致数据丢失。在替换损坏硬件后，系统能自动进行数据冗余备份的补全和负载均衡。存储共享层的MPP数据库，设计运行在冗余磁盘阵列上，并且是具备双控制器的外置磁阵，在确保性能的同时避免了磁阵数据丢失。\uf06c应用分析层应用分析层操作系统和软件模块都部署在冗余磁盘阵列上，系统软件、配置数据等信息在单一磁盘损坏，主机故障等情形下都不会丢失。\n7.4\xa0组网可靠性\nZXVMAX-S产品根据需要，可以通过交换机冗余备份，各主机网络的以太网隧道冗余备份，实现对业务和设备的保护。在单一交换机故障时，备用交换机会及时启用，避免业务中断。在单一的网络传输故障时，备用的传输链路仍然生效，也能避免业务中断。\nSJ-20220623151803-017|2023-03-30（R1.0）\n17'

### 文档分割

由于单个文档的长度往往会超过模型支持的上下文，导致检索得到的知识太长超出模型的处理能力，因此，在构建向量知识库的过程中，我们往往需要对文档进行分割，将单个文档按长度或者按固定的规则分割成若干个 chunk，然后将每个 chunk 转化为词向量，存储到向量数据库中。

在检索时，我们会以 chunk 作为检索的元单位，也就是每一次检索到 k 个 chunk 作为模型可以参考来回答用户问题的知识，这个 k 是我们可以自由设定的。

Langchain 中文本分割器都根据 `chunk_size` (块大小)和 `chunk_overlap` (块与块之间的重叠大小)进行分割。

![image.png](../assets/rag8.png)

* chunk_size 指每个块包含的字符或 Token （如单词、句子等）的数量

* chunk_overlap 指两个块之间共享的字符数量，用于保持上下文的连贯性，避免分割丢失上下文信息

**Langchain 提供多种文档分割方式，区别在怎么确定块与块之间的边界、块由哪些字符/token组成、以及如何测量块大小**

- RecursiveCharacterTextSplitter(): 按字符串分割文本，递归地尝试按不同的分隔符进行分割文本。
- CharacterTextSplitter(): 按字符来分割文本。
- MarkdownHeaderTextSplitter(): 基于指定的标题来分割markdown 文件。
- TokenTextSplitter(): 按token来分割文本。
- SentenceTransformersTokenTextSplitter(): 按token来分割文本
- Language(): 用于 CPP、Python、Ruby、Markdown 等。
- NLTKTextSplitter(): 使用 NLTK（自然语言工具包）按句子分割文本。
- SpacyTextSplitter(): 使用 Spacy按句子的切割文本。

In [12]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
# 知识库中单段文本长度
CHUNK_SIZE = 512

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

In [14]:
# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
# text_splitter.split_text(pdf_page.page_content[0:1000])

In [15]:
split_docs = text_splitter.split_documents(pdf_pages)
print(f"切分后的文件数量：{len(split_docs)}")

切分后的文件数量：55


In [16]:
split_docs[50].page_content

'10\xa0遵循标准与要求\n本章包含如下主题：\n\uf06c 运维标准\n23\n\uf06c 安全标准\n23\n10.1\xa0运维标准\nZXVMAX-S产品符合TMF以下标准：GB923，GB921，GB917，GB922，TR148，TR149\n10.2\xa0安全标准\nZXVMAX-S产品符合以下安全标准：\n\uf06c\nCIS 安全加固标准\n\uf06c\nISO/IEC 15408：信息技术安全评估通用准则（CC，Common Criteria，对应的国标是GB/\nT 18336）\n\uf06c\nGB/T 18336：信息技术 安全技术 信息技术安全性评估准则\n\uf06c\nUL 60950-1：ed.2/CSA-C22.2 No.60950-1-07:ed.2\nSJ-20220623151803-017 | 2023-03-30（R1.0）\n23'

In [17]:
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

切分后的字符数（可以用来大致评估 token 数）：18934


注：如何对文档进行分割，其实是数据处理中最核心的一步，其往往决定了检索系统的下限。但是，如何选择分割方式，往往具有很强的业务相关性——针对不同的业务、不同的源数据，往往需要设定个性化的文档分割方式。因此，在本章，我们仅简单根据 chunk_size 对文档进行分割。对于有兴趣进一步探索的读者，欢迎阅读我们第三部分的项目示例来参考已有的项目是如何进行文档分割的。

## 批量数据预处理

批量处理文件夹中所有文件

In [18]:
import os

# 获取folder_path下所有文件路径，储存在file_paths里
file_paths = []
folder_path = 'data_base/knowledge_path/VMAX-S'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths)

from langchain.document_loaders.pdf import PyMuPDFLoader
# from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:
    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    else:
        print(f"Unsupported file type: {file_type} for file {file_path}")


# 下载文件并存储到text
# 加载所有文档内容到 texts
texts = []
for loader in loaders:
    texts.extend(loader.load())  # 关键步骤：初始化 texts


# 作数据清洗
# 修改后的数据清洗部分（替换原始代码中对应段落）
import re

# 预编译正则表达式（提升效率）
linebreak_pattern = re.compile(
    r'(?<![\\u4e00-\\u9fff])\n(?![\\u4e00-\\u9fff])',  # 负向断言匹配非中文环境换行
    flags=re.DOTALL
)
space_pattern = re.compile(r'[ 　]+')  # 匹配半角/全角空格
special_chars = ['•', '▪', '▫', '▶', '®', '©']  # 可扩展的干扰符号列表

# 替换原始代码中的清洗循环
for text in texts:
    # 1. 清理非中文环境换行
    text.page_content = re.sub(
        linebreak_pattern,
        lambda m: m.group().replace('\n', ''),
        text.page_content
    )
    
    # 2. 批量清理特殊符号
    for char in special_chars:
        text.page_content = text.page_content.replace(char, '')
    
    # 3. 安全删除空格（保留URL等特殊场景）
    text.page_content = space_pattern.sub('', text.page_content)

['data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（5GC业务）.pdf', 'data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.20.80.02）告警处理.pdf', 'data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（语音业务）.pdf', 'data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（数据业务）.pdf', 'data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.20.80.02）故障管理概述.pdf', 'data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（端到端业务）.pdf', 'data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf']


In [19]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [20]:
# 知识库中单段文本长度
CHUNK_SIZE = 512

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

In [21]:
# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
# text_splitter.split_text(text.page_content[0:1000])

In [22]:
split_docs = text_splitter.split_documents(texts)
print(f"切分后的文件数量：{len(split_docs)}")

切分后的文件数量：891


In [23]:
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

切分后的字符数（可以用来大致评估 token 数）：319269


In [24]:
split_docs[300].page_content

'ZXVMAX-S多维价值分析系统告警处理可能原因通过DAPManager查看具体原因并参照其建议处理处理步骤通过DAPManager查看具体原因并参照其建议处理\n1.120114010081ResourceManager进程MEM使用率告警描述\nResourceManager进程MEM使用率告警级别可变（对于不同值，会产生相应级别的告警）可能原因内存配置无法满足业务需求。内存使用率达到上限。处理步骤\n1.修改告警阈值，当内存使用率小于或等于阈值时，告警恢复；\n2.对节点进行内存扩容；\n3.收集故障信息，通过菜单日志搜索yarn服务日志，联系技术支持分析日志；\n1.121114020039NodeManager进程本地空间使用率告警描述\nNodeManager进程本地空间使用率告警级别可变（对于不同值，会产生相应级别的告警）可能原因\nYarn空间使用率超过空间告警阈值处理步骤\n1.修改告警阈值，当空间使用率小于或等于阈值时，告警恢复；\n2.对节点进行磁盘扩容\n66\nSJ-20220623151803-011|2022-06-20（R1.0）'

# 向量数据库搭建

### 安装Milvus数据库

Milvus Standalone 是单机服务器部署。Milvus Standalone 的所有组件都打包到一个[Docker 镜像](https://milvus.io/docs/install_standalone-docker.md)中，部署起来非常方便。如果你有生产工作负载，但又不想使用 Kubernetes，那么在内存充足的单机上运行 Milvus Standalone 是一个不错的选择。此外，Milvus Standalone 通过主从复制支持高可用性。

**使用 Docker Compose 运行 Milvus (Linux)**

Milvus 在 Milvus 资源库中提供了 Docker Compose 配置文件。要使用 Docker Compose 安装 Milvus，只需运行

```shell
# Download the configuration file
$ wget https://bgithub.xyz/milvus-io/milvus/releases/download/v2.5.6/milvus-standalone-docker-compose.yml -O docker-compose.yml

# Start Milvus
$ sudo docker compose up -d

Creating milvus-etcd  ... done
Creating milvus-minio ... done
Creating milvus-standalone ... done
```

- 如果运行上述命令失败，请检查系统是否安装了 Docker Compose V1。如果是这种情况，建议你根据[本页](https://docs.docker.com/compose/)的说明迁移到 Docker Compose V2。
- 如果您在拉取镜像时遇到任何问题，请通过[community@zilliz.com](mailto:community@zilliz.com)联系我们，并提供有关问题的详细信息，我们将为您提供必要的支持。

启动 Milvus 后、

- 名为milvus-standalone、milvus-minio和milvus-etcd的容器启动。
  - **milvus-etcd**容器不向主机暴露任何端口，并将其数据映射到当前文件夹中的**volumes/etcd**。
  - **milvus-minio**容器使用默认身份验证凭据在本地为端口**9090**和**9091**提供服务，并将其数据映射到当前文件夹中的**volumes/minio**。
  - **Milvus-standalone**容器使用默认设置为本地**19530**端口提供服务，并将其数据映射到当前文件夹中的**volumes/milvus**。

你可以使用以下命令检查容器是否启动并运行：

```shell
$ sudo docker-compose ps

      Name                     Command                  State                            Ports
--------------------------------------------------------------------------------------------------------------------
milvus-etcd         etcd -advertise-client-url ...   Up             2379/tcp, 2380/tcp
milvus-minio        /usr/bin/docker-entrypoint ...   Up (healthy)   9000/tcp
milvus-standalone   /tini -- milvus run standalone   Up             0.0.0.0:19530->19530/tcp, 0.0.0.0:9091->9091/tcp
```

你还可以访问 Milvus WebUI，网址是`http://127.0.0.1:9091/webui/` ，了解有关 Milvus 实例的更多信息。详情请参阅[Milvus WebUI](https://milvus.io/docs/zh/milvus-webui.md)。

**停止和删除 Milvus**

您可以按以下步骤停止和删除此容器

```shell
# Stop Milvus
$ sudo docker compose down

# Delete service data
$ sudo rm -rf volumes
```

**Attu（Milvus 图形用户界面）**

https://github.com/zilliztech/attu

```
docker run -p 9092:3000 -e MILVUS_URL=129.201.70.35:19530 zilliz/attu:v2.6
```



**milvus使用指南**

- [数据库](https://milvus.io/docs/zh/manage_databases.md)
- Collections
- Schema 和数据字段
- 插入和删除
- 索引
- 搜索和 Rerankers

> 参考https://milvus.io/docs/zh/manage_databases.md


## 前序数据预处理

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# 批量处理文件夹中所有文件
import os

# 获取folder_path下所有文件路径，储存在file_paths里
file_paths = []
folder_path = 'data_base/knowledge_path/VMAX-S'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths)

from langchain.document_loaders.pdf import PyMuPDFLoader
# from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:
    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    else:
        print(f"Unsupported file type: {file_type} for file {file_path}")

# 下载文件并存储到text
# 加载所有文档内容到 texts
texts = []
for loader in loaders:
    texts.extend(loader.load())  # 关键步骤：初始化 texts

    
# 作数据清洗
# 修改后的数据清洗部分（替换原始代码中对应段落）
import re

# 预编译正则表达式（提升效率）
linebreak_pattern = re.compile(
    r'(?<![\\u4e00-\\u9fff])\n(?![\\u4e00-\\u9fff])',  # 负向断言匹配非中文环境换行
    flags=re.DOTALL
)
space_pattern = re.compile(r'[ 　]+')  # 匹配半角/全角空格
special_chars = ['•', '▪', '▫', '▶', '®', '©']  # 可扩展的干扰符号列表

# 替换原始代码中的清洗循环
for text in texts:
    # 1. 清理非中文环境换行
    text.page_content = re.sub(
        linebreak_pattern,
        lambda m: m.group().replace('\n', ''),
        text.page_content
    )

    # 2. 批量清理特殊符号
    for char in special_chars:
        text.page_content = text.page_content.replace(char, '')

    # 3. 安全删除空格（保留URL等特殊场景）
    text.page_content = space_pattern.sub('', text.page_content)




#导入文本分割器
from langchain_text_splitters import RecursiveCharacterTextSplitter
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''


# 知识库中单段文本长度
CHUNK_SIZE = 512

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50


# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)


split_docs = text_splitter.split_documents(texts)
print(f"切分后的文件数量：{len(split_docs)}")



print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")



split_docs[300].page_content



预期输出：
```bash
['data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（5GC业务）.pdf', 'data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.20.80.02）告警处理.pdf', 'data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（语音业务）.pdf', 'data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（数据业务）.pdf', 'data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.20.80.02）故障管理概述.pdf', 'data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（端到端业务）.pdf', 'data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf']
切分后的文件数量：891
切分后的字符数（可以用来大致评估 token 数）：319269
'ZXVMAX-S多维价值分析系统告警处理可能原因通过DAPManager查看具体原因并参照其建议处理处理步骤通过DAPManager查看具体原因并参照其建议处理\n1.120114010081ResourceManager进程MEM使用率告警描述\nResourceManager进程MEM使用率告警级别可变（对于不同值，会产生相应级别的告警）可能原因内存配置无法满足业务需求。内存使用率达到上限。处理步骤\n1.修改告警阈值，当内存使用率小于或等于阈值时，告警恢复；\n2.对节点进行内存扩容；\n3.收集故障信息，通过菜单日志搜索yarn服务日志，联系技术支持分析日志；\n1.121114020039NodeManager进程本地空间使用率告警描述\nNodeManager进程本地空间使用率告警级别可变（对于不同值，会产生相应级别的告警）可能原因\nYarn空间使用率超过空间告警阈值处理步骤\n1.修改告警阈值，当空间使用率小于或等于阈值时，告警恢复；\n2.对节点进行磁盘扩容\n66\nSJ-20220623151803-011|2022-06-20（R1.0）'
```

## embedding模型准备z

In [2]:
# ZhipuAIEmbeddings模型
# import os
# from dotenv import load_dotenv, find_dotenv

# _ = load_dotenv(find_dotenv())

# from langchain_community.embeddings import ZhipuAIEmbeddings

# my_emb = ZhipuAIEmbeddings(
#     model="embedding-2",
#     api_key = os.environ['ZHIPUAI_API_KEY'],
# )

In [3]:
# from langchain_ollama.embeddings import OllamaEmbeddings

# # 初始化嵌入模型
# my_emb = OllamaEmbeddings(
#     base_url='http://localhost:11434',
#     model="bge-m3:latest"
# )

In [ ]:
from langchain_community.embeddings import XinferenceEmbeddings

# 使用API代理服务提高访问稳定性
my_emb = XinferenceEmbeddings(
    server_url="http://129.201.70.35:9997", 
    model_uid="my_qwen3_embed_0.6b"
)


## 构建Milvus向量库

In [ ]:
import os
from langchain_community.vectorstores import Milvus
from langchain_core.documents import Document


# 创建嵌入模型
my_emb = XinferenceEmbeddings(
    server_url="http://129.201.70.35:9997", 
    model_uid="my_qwen3_embed_0.6b"
)

# 向量库创建
# connection_args = {
#     "uri": "tcp://129.201.70.35:19530"
# }

connection_args = {
    "host": "129.201.70.35",
    "port": 19530,
}

# 定义每批处理的文档数量
batch_size = 30

# 如果只想导入部分数据
# split_docs = split_docs[:3]
try:
    # 计算总批次数
    total_batches = (len(split_docs) + batch_size - 1) // batch_size
    
    # 初始化向量数据库（如果是第一次创建）
    vectordb = None
    
    for batch_num in range(total_batches):
        # 计算当前批次的起始和结束索引
        start_idx = batch_num * batch_size
        end_idx = min((batch_num + 1) * batch_size, len(split_docs))
        
        # 获取当前批次的文档
        batch_docs = split_docs[start_idx:end_idx]
        
        print(f"正在处理第 {batch_num + 1}/{total_batches} 批文档 (文档 {start_idx}-{end_idx-1})")

        if batch_num == 0:
            # 第一次创建向量数据库
            vectordb = Milvus.from_documents(
            documents=batch_docs,
            embedding=my_emb,
            collection_name="ZXVMAXS1204",
            drop_old=False,
            connection_args=connection_args,
            )
            
            # 如果使用Milvus的混合检索
            # 创建 Milvus VectorStore，实现 dense + sparse 混合检索 
            # vectordb = Milvus.from_documents( 
            # documents=batch_docs, 
            # embedding=my_emb, # 用于语义检索的 dense 向量 
            # builtin_function=BM25BuiltInFunction(), # BM25 的 sparse 全文检索 
            # vector_field=["dense", "sparse"], # 指定两个向量字段名称 
            # connection_args=connection_args, 
            # collection_name="ZXVMAXS6",
            # consistency_level="Strong", 
            # drop_old=True, # 若已有旧 collection，可删除重建 
            # )

        else:
            # 后续批次添加到现有集合
            vectordb.add_documents(batch_docs)
        
        # 每批处理后持久化
        # vectordb.persist()
        print(f"第 {batch_num + 1} 批文档已成功导入并持久化")
    
    print("所有文档已成功导入并持久化到向量数据库。")
    
except Exception as e:
    print(f"处理过程中发生错误: {e}")

## 连接数据库进行测试

In [ ]:
from langchain_community.vectorstores import Milvus

my_emb = XinferenceEmbeddings(
    server_url="http://129.201.70.35:9997", 
    model_uid="my_qwen3_embed_0.6b"
)

# connection_args = {
#     "uri": "tcp://129.201.70.35:19530"
# }

connection_args = {
    "host": "129.201.70.35",
    "port": 19530,
}
# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="ZXVMAXS1204",  # Milvus 集合名称
        connection_args=connection_args,
    )




In [ ]:
from pymilvus import connections, utility

def list_all_collections():
    """获取所有collection列表"""
    try:
        # 连接到Milvus（使用与vectordb相同的参数）
        connections.connect(
            alias="default",
            host="129.201.70.35",
            port="19530"
        )
        
        # 获取所有collection
        collections = utility.list_collections()
        print("所有Collection列表:")
        for i, coll_name in enumerate(collections, 1):
            print(f"{i}. {coll_name}")
            
        return collections
        
    except Exception as e:
        print("获取collection列表失败:", e)
        return []
    finally:
        # 断开连接
        connections.disconnect("default")

# 执行
collections = list_all_collections()
print(f"总共找到 {len(collections)} 个collection")

In [ ]:
# 测试相似性搜索
query = "ZXVMAXS的5G上网日志有那些功能？"
results = vectordb.similarity_search(query, k=2)
print("相似性搜索结果:")
for i, doc in enumerate(results):
    print(f"结果 {i+1}: {doc.page_content} - 元数据: {doc.metadata}")

In [ ]:
# 获取带相似度分数的结果
results_with_score = vectordb.similarity_search_with_score(query, k=3)
print("带分数的搜索结果:")
for doc, score in results_with_score:
    print(f"内容: {doc.page_content}, 分数: {score:.4f}")

# 构建检索问答链

我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

### Milvus

In [ ]:
from langchain_community.vectorstores import Milvus

# ollama版本
# from langchain_community.embeddings import OllamaEmbeddings
# my_emb = OllamaEmbeddings(base_url='http://129.201.70.35:11434', model="dengcao/Qwen3-Embedding-0.6B:F16")

# xinference版本
from langchain_community.embeddings import XinferenceEmbeddings
my_emb = XinferenceEmbeddings(
    server_url="http://129.201.70.35:9997", 
    model_uid="my_qwen3_embed_0.6b"
)

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="ZXVMAXS1204",  # Milvus 集合名称
        connection_args={
            "host": "129.201.70.35",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )

In [ ]:
results = vectordb.similarity_search(query="什么是vmax的上网日志系统？", k=2)
results

预期输出： 
```
[Document(metadata={'pk': 461017040453077208, 'producer': 'Apache FOP Version 2.6', 'creator': 'DITA Open Toolkit', 'creationdate': '2023-05-23T21:45:33+08:00', 'source': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'file_path': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'total_pages': 29, 'format': 'PDF 1.4', 'title': '目录', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20230523214533+08'00'", 'page': 13}, page_content='4\xa0功能本章包含如下主题：\uf06c上网日志保存\n10\n\uf06c上网日志查询\n10\n\uf06c上网日志批量导入查询\n11\n\uf06c日志管理\n11\n\uf06c账号管理\n11\n\uf06c角色管理\n12\n\uf06c资源监控\n12\n\uf06c告警管理\n12\n\uf06c省级网关对接\n12\n\uf06c拨测结果自动比对功能\n12\n\uf06cNAT日志入库功能\n13\n\uf06c北向接口\n13\n\uf06c上网日志历史查询\n13\n\uf06c云化上网日志XDR查询\n13\n以下介绍ZXVMAX-S的主要的功能。\n4.1\xa0上网日志保存\uf06c支持使用Gbase数据库或HDFS保存上网日志。\uf06c上网日志保存时间可配置，最短保存7天时间，最长可保存一年时间。\uf06c支持自动清理超过保存时间的上网日志。\n4.2\xa0上网日志查询可通过web界面指定查询条件，查询用户上网日志。支持的查询条件：\uf06c时间范围+公网IP\n\uf06c时间范围+目的IP\n\uf06c时间范围+MSISDN\n\uf06c时间范围+IMSI\n\uf06c时间范围+URL\n\uf06c支持组合以上五种基础条件，或在基础条件上增加指定公网端口、目的端口，做更精确的查询。\n10\nSJ-20220623151803-017|2023-03-30（R1.0）'),
 Document(metadata={'pk': 461017040453076907, 'producer': 'Apache FOP Version 2.3', 'creator': 'DITA Open Toolkit', 'creationdate': '2022-06-23T16:34:22+08:00', 'source': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.20.80.02）告警处理.pdf', 'file_path': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.20.80.02）告警处理.pdf', 'total_pages': 330, 'format': 'PDF 1.4', 'title': '目录', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20220623163422+08'00'", 'page': 297}, page_content='ZXVMAX-S多维价值分析系统告警处理可能原因确保实际填充率达到预期填充率处理步骤确保实际填充率达到预期填充率\n7.2194000200303HTTP接口下行TCP重传报文数完整率(小时)告警描述确保实际填充率达到预期填充率告警级别警告可能原因确保实际填充率达到预期填充率处理步骤确保实际填充率达到预期填充率\n7.2204000200304HTTP接口TCP建链响应时延（ms）完整率(小时)告警描述确保实际填充率达到预期填充率告警级别警告可能原因确保实际填充率达到预期填充率处理步骤确保实际填充率达到预期填充率\n7.2214000200305HTTP接口TCP建链确认时延（ms）完整率(小时)告警描述确保实际填充率达到预期填充率\n276\nSJ-20220623151803-011|2022-06-20（R1.0）')]
```

## 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

### ollama版本

In [ ]:
# ollama版本
from langchain_community.llms import Ollama
my_llm = Ollama(base_url='http://localhost:11434', model='qwen3:8B', temperature=0)
my_llm.invoke("你是谁？/no_think")

### vllm版本

In [ ]:
from langchain_openai import ChatOpenAI

# 连接信息 - 注意这里的 base_url 需要与你的vLLM服务器地址一致
my_llm = ChatOpenAI(
    openai_api_key="token-abc123",  # 若服务端无验证，可设为"EMPTY"等任意非空字符串
    base_url="http://129.201.70.35:9991/v1",  # 你的vLLM服务器地址
    model_name="my_qwen3_8b",
    temperature=0,
    # streaming=True  # 同样支持流式输出
)

# 调用模型，返回的是AIMessage对象，通过.content获取文本内容
result = my_llm.invoke("你是谁？/no_think")
print(result.content)

In [ ]:
# 避雷！VLLMOpenAI这个方法问题很多！幻觉严重
# from langchain_community.llms import VLLMOpenAI  
# my_llm = VLLMOpenAI(
#     openai_api_key="token-abc123",          
#     openai_api_base="http://129.201.70.35:9991/v1",  
#     model_name="my_qwen3_8b",  
#     # max_tokens=10240,                
#     temperature=0.5,  
#     streaming=True                
# )
# my_llm.invoke("你是谁？/no_think")

### Xinference版本

In [ ]:
# Xinference版本
from langchain_community.llms import Xinference
my_llm = Xinference(
    server_url="http://129.201.70.35:9997",
    model_uid = "my_qwen3_14b", 
    temperature=0
)
my_llm.invoke("你是谁？/no_think")

预期输出： 

```
'<think>\n嗯，用户发来“你好”，我需要回应。首先，要友好，用中文。然后，可能用户想开始对话，所以应该主动询问他们有什么需要帮助的。保持自然，不要太机械。比如可以说“你好！有什么我可以帮你的吗？”或者更亲切一点，比如“你好呀！今天过得怎么样？有什么我可以协助你的吗？”这样既友好又开放，鼓励用户进一步交流。同时要注意语气，避免过于正式，让对话更轻松。另外，可能用户只是测试，所以需要保持灵活，根据后续回复调整。总之，回应要简洁、友好，引导用户说出具体需求。\n</think>\n\n你好呀！今天过得怎么样？有什么我可以协助你的吗？😊'
```

## 构建检索问答链

prompts

In [ ]:
from langchain_core.prompts import PromptTemplate


# 优化后的提示模板
template = """你是VMAX运维助手，请严格根据以下上下文信息回答问题。

上下文：
{context}

问题：{question}

请根据上下文直接回答，如果上下文没有提供答案，请说"我不知道"。
请确保回答的连贯性和唯一性，不要重复相同的信息。
最后说"谢谢你的提问！"

回答："""

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)


#### 创建一个基于模板的检索链： 基础检索版本

In [ ]:
from langchain.chains import RetrievalQA
# 优化检索器
base_retriever = vectordb.as_retriever(
    search_kwargs={
        "k": 8,  # 减少返回文档数，提高相关性
        "score_threshold": 0.75  # 相似度阈值
    },
    search_type="mmr"
)

# 创建问答链
qa_chain = RetrievalQA.from_chain_type(
    my_llm,
    retriever=base_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

## 检索问答链效果测试

### 基于召回结果和 query 结合起来构建的 prompt 效果

In [ ]:
# 测试
question_1 = "什么是ZXVMAX-S的5GC业务？"
result = qa_chain.invoke({"query": question_1})  # 使用invoke而非call
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])
# print("\n来源文档数量：", len(result["source_documents"]))

预期输出： 
```
大模型+知识库后回答 question_1 的结果：


ZXVMAX-S的5GC业务是指其针对第五代核心网（5GC）场景设计的运维分析能力，涵盖信令回溯、记录查询、呼叫分析、网络质量分析、业务质量定界、时延分析等功能，支持对用户投诉问题的快速定位（如4G/5G无线、核心网、SP及用户因素分析），以及通过区域感知和用户感知功能对全网或重点区域的性能评估与根因定位。系统采用分层架构（采集层、存储共享层、应用分析层），依赖探针设备采集原始数据并生成XDR格式，为网络优化和故障处理提供支撑。

谢谢你的提问！
```

In [ ]:
question_2 = "周杰伦是谁？/no_think"
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

预期输出： 

```
大模型+知识库后回答 question_2 的结果：


我是一个VMAX运维助手，专注于提供与VMAX系统相关的技术支持和信息。关于周杰伦的问题，我无法提供相关信息。谢谢你的提问！
```

### 无知识库大模型自己回答的效果

In [ ]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
my_llm.invoke(prompt_template)

预期输出：
```
'<think>\n\n</think>\n\nZXVMAX-S 是华为推出的一款面向5G网络的虚拟化核心网（5GC）解决方案，其全称是 **ZXVMAX-S 5G Core**。它是一种基于云原生架构的5G核心网业务平台，支持多种5GC业务功能，如：\n\n- **UE（用户设备）管理（AMF）**\n- **会话管理（SMF）**\n- **接入和移动性管理（AMF）**\n- **策略控制（PCF）**\n- **网络切片管理（NSSF）**\n- **统一数据管理（UDM）**\n- **服务化接口（NRF）**\n- **策略控制（PCF）**\n- **网络暴露功能（NEF）**\n\n### ZXVMAX-S 的5GC业务特点：\n\n1. **云原生架构**：基于微服务架构，支持灵活扩展和高可用性。\n2. **多业务支持**：支持多种5G业务场景，如eMBB（增强移动宽带）、URLLC（超可靠低时延通信）、mMTC（大规模机器通信）等。\n3. **开放接口**：支持与第三方应用、网络切片管理平台、业务链管理平台等进行对接。\n4. **灵活部署**：支持云化部署（如NFVI）、虚拟化部署（如VNF）和混合部署。\n5. **高可靠性**：支持多实例冗余、故障自动切换、负载均衡等机制，保障业务连续性。\n6. **支持网络切片**：支持不同业务场景下的网络切片管理，满足不同行业对网络性能、安全、隔离等的不同需求。\n\n### 应用场景：\n\n- **工业互联网**：支持高可靠、低时延的工业自动化控制。\n- **车联网（V2X）**：支持超低时延和高可靠性的车联网通信。\n- **智慧城市**：支持大规模设备接入和数据处理。\n- **远程医疗**：支持高带宽、低时延的远程手术和诊断。\n- **AR/VR**：支持高带宽、低时延的沉浸式体验。\n\n### 总结：\n\nZXVMAX-S 的5GC业务是华为面向5G时代推出的核心网解决方案，支持多种5G业务功能和场景，具备高可靠性、灵活性和开放性，适用于多种行业和应用场景，是构建5G网络的重要基础设施。'
```


In [ ]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
my_llm.invoke(prompt_template)

预期输出： 

```
'<think>\n\n</think>\n\n周杰伦（Jay Chou），1970年1月18日出生于中国台湾省新北市，是中国台湾的著名音乐人、歌手、音乐制作人、导演和演员。他是华语乐坛最具影响力的音乐人之一，以其独特的音乐风格、创新的编曲方式和深情的歌词而闻名。\n\n周杰伦的音乐融合了多种元素，包括流行、R&B、嘻哈、中国风、古典音乐等，他的作品常常带有浓厚的东方文化色彩。他不仅在音乐创作上表现出色，还涉足电影导演领域，执导了多部电影，如《不能说的秘密》《天台上的魔幻时刻》等，均获得广泛好评。\n\n周杰伦的代表作品包括《七里香》《青花瓷》《菊花台》《告白气球》《听妈妈的话》等，这些歌曲不仅在华语地区广为传唱，也在全球范围内拥有大量粉丝。他多次获得金曲奖等多项音乐大奖，是华语乐坛的标志性人物之一。\n\n总的来说，周杰伦不仅是一位才华横溢的音乐人，也是华语流行文化的重要代表人物。'
```

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 添加历史对话的记忆功能

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
# 修改后的Prompt模板（添加chat_history变量）
template = """你是VMAX运维助手，请参考以下对话历史和上下文来回答问题：
    {chat_history}
    
    相关上下文：
    {context}
    
    问题：{question}
    回答结束时说“谢谢你的提问！”
    """
    
QA_PROMPT = PromptTemplate(
        input_variables=["chat_history", "context", "question"],
        template=template
    )
    
# 创建对话链
qa_chain = ConversationalRetrievalChain.from_llm(
        llm=my_llm,
        retriever=vectordb.as_retriever(),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": QA_PROMPT},
        chain_type="stuff"
    )
    
    # result = qa_chain({"question": question})
    # return result["answer"]
    

然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [ ]:
questions = [
    "什么是ZXVMAX-S的5GC业务？/no_think",
    "包含哪些功能/no_think", 
    "整理成markdown表格形式/no_think" 
]

for question in questions:
    result = qa_chain({"question": question})  # Pass string directly, not dict
    # print(f"问题：{question}")
    print(f"回答：{result['answer']}")
    # print("对话历史：", memory.load_memory_variables({}))
    print("\n" + "="*50 + "\n")

预期输出：

```

回答：

ZXVMAX-S的5GC业务是指中兴通讯推出的ZXVMAX-S多维价值分析系统在5G核心网（5GC）场景下的应用。该系统支持对5GC网络中的各种业务数据进行分析，包括性能分析、事件分析、终端分析和业务优化等，帮助用户更好地理解和优化5G网络的运行效率与服务质量。系统采用分层架构，包括采集层、存储共享层和应用分析层，能够根据用户需求进行数据自定义查询和分析，满足不同场景下的运维与管理需求。

谢谢你的提问！

==================================================

回答：

ZXVMAX-S的5GC业务包含以下功能：

1. **性能分析**：对5GC网络中的性能数据进行分析，帮助识别网络瓶颈，优化网络运行效率。
2. **事件分析**：对5GC网络中发生的各类事件进行分析，支持快速定位和处理问题。
3. **终端分析**：分析终端设备在5GC网络中的行为和表现，提升用户体验。
4. **业务优化**：针对5GC网络中的业务进行优化，提升服务质量与业务运行效率。

此外，ZXVMAX-S系统支持根据用户需求进行数据自定义查询和分析，并且可以根据目标用户的需求开启对应权限，部署相应的模块，以满足不同场景下的运维与管理需求。

谢谢你的提问！

==================================================

回答：

以下是ZXVMAX-S的5GC业务功能整理成的Markdown表格形式：

```markdown
| 功能模块       | 功能描述                                                                 |
|----------------|--------------------------------------------------------------------------|
| 性能分析       | 对5GC网络中的性能数据进行分析，帮助识别网络瓶颈，优化网络运行效率。       |
| 事件分析       | 对5GC网络中发生的各类事件进行分析，支持快速定位和处理问题。               |
| 终端分析       | 分析终端设备在5GC网络中的行为和表现，提升用户体验。                       |
| 业务优化       | 针对5GC网络中的业务进行优化，提升服务质量与业务运行效率。                 |
| 数据自定义查询 | 支持根据用户需求进行数据自定义查询和分析，满足不同场景下的运维与管理需求。 |
| 权限管理       | 根据目标用户的需求，开启对应权限，部署相应的模块。                         |
```

谢谢你的提问！

==================================================
```